Installing the necessary packages

In [1]:
#package to help us get the movie names for our dataset
!pip install imdbpy
!pip install scrapy
!pip install bs4 selenium

  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-py3-none-any.whl size=12552 sha256=b00e246b736b14793f8fc5f1bd7eeb0572362580db8a4ca437da3cf5699b6f5a
  Stored in directory: c:\users\lenovo\appdata\local\pip\cache\wheels\dc\d0\bf\0cc715c01fce0bace63b46283acf5cc630d5e5dbb4602c54e5
  Created wheel for protego: filename=Protego-0.1.16-py3-none-any.whl size=7769 sha256=a8999991e39fe8f50c139a9187f48111222dbeb9b006028f7b66a41cdf181286
  Stored in directory: c:\users\lenovo\appdata\local\pip\cache\wheels\ca\44\01\3592ccfbcfaee4ab297c4097e6e9dbe1c7697e3531a39877ab
Successfully built PyDispatcher protego


importing the necessary packages

In [2]:
import imdb 
from requests import get
from bs4 import BeautifulSoup
import re
import pandas as pd


from selenium import webdriver      
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import time

Getting a list of top 250 movies from IMDB

In [3]:
def imdb_movie_list():
    ia = imdb.IMDb()
    search = ia.get_top250_movies()
    a=[]
    for title in range (len(search)):
        a.append(search[title]['title'])
  
    return a

Creating a function to extract exact movie URL once we reach the search window


* This URL function will help us get the unique movie title code corresponding to each movie within imdb servers




In [4]:
def movie_url_funct(movie_url):
    response = get(movie_url)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    result_container = html_soup.find_all('table', class_ = 'findList')
    top_result=result_container[0]
    top_result=top_result.find('td', class_ = 'result_text')
    return ('https://www.imdb.com'+top_result.a.get('href'))

Once we extracted the movie id and urls from the function above we can use this function to jump to all user reviews page

In [5]:
version_regex = re.compile(r"\/title\/[a-zA-Z0-9]\w+\/reviews")# regex to extract all user reviews href link

def all_reviews_url_funct(true_movie_url):
    response=get(true_movie_url)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    review_result_container = html_soup.findAll('div',class_='user-comments')# this class has the user review tags
    x=review_result_container[0]
    all_reviews_url=x.findAll('a') # to find all the <a> sections which contains the href parameters

    for x in all_reviews_url:
        a=x.get('href')
        found=False
        for match in version_regex.finditer(a):
            return ('https://www.imdb.com/'+ match.group())
            found=True
        if not found:
              pass;

Once you have the all user review page, now we can start extracting the reviews from this page :D
Few things to keep in mind are:



*   We will use a selenium based option combined with beautiful soup to help click on the load more button and extract all the reviews which we want.
*   we need to extract keeping in mind that the api calls are not too frequent, we'll implement some logic that embeds sleep timer in the code





**Intializing the selenium browser which will extract everything on our behalf**

to download the chromedriver please visit the link and download the version which corresponds to your current google chrome browser:
https://sites.google.com/a/chromium.org/chromedriver/home

In [6]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
driver = webdriver.Chrome(r"E:\chromedriver.exe", options=options,)

Defining the function that keeps clicking the load more button till the entire review page has loaded and then fetches the page source of the final page, which in turn gives us all the user reviews corresponding to the entered review URL

In [21]:
 
def pagesource_final(url_name,load_more_button_xpath):
    driver.get(url_name)
    time.sleep(2)
    while True:        
        try:
            loadMoreButton = driver.find_element_by_xpath(load_more_button_xpath)
            time.sleep(2)
            driver.execute_script("window.scrollTo(0, window.scrollY + 100)") #scrolls the page in case the button is no available yet
            driver.implicitly_wait(3)
            ActionChains(driver).move_to_element(loadMoreButton).click(loadMoreButton).perform()#clicks the load more button
            time.sleep(3)
            page_source=driver.page_source
    #         driver.get(YOUTUBER_HOME_PAGE_URL)   
        except Exception as e:
            page_source=driver.page_source
    #             print(e)
            break
    return page_source
    print ("Complete")

Creating the function that will finally create our Reviews dataset based on the page source extracted above

In [95]:
def imdb_review_dataset_creator(page_source):
    soup = BeautifulSoup(page_source, "html.parser")
    #Get the title of the movie
    all = soup.find(id="main")
    parent = all.find(class_ ="parent")
    name = parent.find(itemprop = "name")
    # url = name.find(itemprop = 'url')
    # film_title = url.get_text()

    #get the movie name
    movie_name=parent.a.text.replace("\n", "")

    #get the movie year
    year=parent.span.text.replace(" ", "").replace("\n","").replace("(","").replace(")","")

    #Get the title of the review
    title_rev = all.select(".title")
    title = [t.get_text().replace("\n", "") for t in title_rev]


    #Get the review
    review_rev = all.select(".content .text")
    review = [r.get_text() for r in review_rev]
    
    table_review = pd.DataFrame({
    "Movie" : movie_name,
    "Year Released": year,
    "Review Title" : title,
    "Review" : review
    })
    
    return  table_review
                    

Calling all the functions to create our much awaited dataset

This functions gets the list of top 250 movies from IMDB
you can basically have any movie name of your choice within this list

In [9]:
movie_list=imdb_movie_list()
movie_list = [x.replace(' ','+') for x in movie_list] #adding a + instead of whitespace since that's how IMDB parses URL
search_url_list=['https://www.imdb.com/find?q='+x for x in movie_list]# creating IMDB URLs for each movie list we fethced abovesearch_url_list=[x+'&s=tt&ref_=fn_tt' for x in search_url_list]

calling the function to get unique movie title for each movie keyword we send

In [10]:
movie_title_url=[]
for movie_url in search_url_list:
    movie_title_url.append(movie_url_funct(movie_url))

Once we have the unique title url for each movie we extract the URL that takes us to user review page for each movie

In [11]:
user_review_url=[]
for unique_movie_url in movie_title_url:
    user_review_url.append(all_reviews_url_funct(unique_movie_url))

Calling the function that clicks on load more button and finally stores the page source in a variable, we then use that final page source to extract the info we need and append it to our dataframe

In [29]:
page_source=''
page_source_list=[]
load_more_button_xpath='//*[@id="load-more-trigger"]'

for review_url in user_review_url:
    page_source_list.append(pagesource_final(review_url,load_more_button_xpath))


**Declaring the DataFrame**

In [38]:
#Make it into dataframe
IMDB_dataset_creator = pd.DataFrame(columns={
    "Movie" ,
    "Year Released",
    "Review Title",
    "Review"
})

In [110]:
for page_source in page_source_list:
    a=imdb_review_dataset_creator(page_source)
    IMDB_dataset_creator=IMDB_dataset_creator.append(a,ignore_index = True)